In [1]:
# Import Block
import cv2
import numpy as np
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import distance
from sklearn.cluster import DBSCAN


img =cv2.imread('braille_page_4.jpg')

In [2]:
# Nepali Braille 6-dot encoding map
# Dot numbering:
# 1 4
# 2 5
# 3 6

def dots_to_bit(dots):
    """Convert list of active dots (1–6) to a 6-bit integer."""
    val = 0
    for d in dots:
        val |= 1 << (6 - d)
    return val

braille_map = {
    dots_to_bit([1]): 'अ',
    dots_to_bit([3, 4, 5]): 'आ',
    dots_to_bit([2, 4]): 'इ',
    dots_to_bit([3, 5]): 'ई',
    dots_to_bit([1, 3, 6]): 'उ',
    dots_to_bit([1, 2, 5, 6]): 'ऊ',
    dots_to_bit([1, 5, 6]): 'ऋ',
    dots_to_bit([1, 5]): 'ए',
    dots_to_bit([3, 4]): 'ऐ',
    dots_to_bit([1, 3, 5]): 'ओ',
    dots_to_bit([2, 4, 6]): 'औ',
    dots_to_bit([1, 6]): 'अं',
    dots_to_bit([6]): 'अः',

    dots_to_bit([1, 3]): 'क',
    dots_to_bit([4, 6]): 'ख',
    dots_to_bit([1, 2, 4, 5]): 'ग',
    dots_to_bit([1, 2, 6]): 'घ',
    dots_to_bit([3, 4, 6]): 'ङ',
    dots_to_bit([1, 4]): 'च',
    dots_to_bit([1, 6]): 'छ',
    dots_to_bit([2, 4, 5]): 'ज',
    dots_to_bit([3, 5, 6]): 'झ',
    dots_to_bit([2, 5]): 'ञ',
    dots_to_bit([2, 3, 4, 6]): 'ट',
    dots_to_bit([2, 4, 5, 6]): 'ठ',
    dots_to_bit([1, 2, 4, 6]): 'ड',
    dots_to_bit([1, 2, 3, 4, 6]): 'ढ',
    dots_to_bit([3, 4, 5, 6]): 'ण',
    dots_to_bit([2, 3, 4, 5]): 'त',
    dots_to_bit([1, 4, 5, 6]): 'थ',
    dots_to_bit([1, 4, 5]): 'द',
    dots_to_bit([2, 3, 5, 6]): 'ध',
    dots_to_bit([1, 3, 4, 5]): 'न',
    dots_to_bit([1, 2, 3, 4]): 'प',
    dots_to_bit([2, 3, 5]): 'फ',
    dots_to_bit([1, 2]): 'ब',
    dots_to_bit([4, 5]): 'भ',
    dots_to_bit([1, 3, 4]): 'म',
    dots_to_bit([1, 3, 4, 5, 6]): 'य',
    dots_to_bit([1, 2, 3, 5]): 'र',
    dots_to_bit([1, 2, 3]): 'ल',
    dots_to_bit([1, 2, 3, 6]): 'व',
    dots_to_bit([1, 4, 6]): 'श',
    dots_to_bit([1, 2, 3, 4, 6]): 'ष',
    dots_to_bit([2, 3, 4]): 'स',
    dots_to_bit([1, 2, 5]): 'ह',
    dots_to_bit([1, 2, 3, 4, 5]): 'क्ष',
    dots_to_bit([1, 5, 6]): 'ज्ञ',
}

# Special compound (multi-cell)
braille_map_compound = {
    # 'त्र' uses two cells: [5] + [1,2,4,5,6]
    (dots_to_bit([5]), dots_to_bit([1, 2, 4, 5, 6])): 'त्र',
}

In [3]:
# preprocessing Block
def preprocess_braille_image(
    img_gray,
    method="adaptive",   # "adaptive", "sauvola", or "otsu"
    block_size=31,       # for adaptive/local threshold (odd, e.g. 25–51)
    C=10,                # constant for adaptive/local threshold
    morph_open=3,        # kernel size for opening (3–7)
    morph_close=5,       # kernel size for closing (3–7)
    apply_tophat=False,  # True if embossing subtle
    min_blob_area=5      # small debris threshold
):
    """
    Preprocess Braille grayscale image for dot detection.
    Returns binary image (dots = white, background = black).
    """

    # --- Step 1: Denoise slightly (optional but helps) ---
    img_blur = cv2.GaussianBlur(img_gray, (3, 3), 0)

    # --- Step 2: Binarization ---
    if method == "adaptive":
        bin_img = cv2.adaptiveThreshold(
            img_blur, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,  # invert → dots white on black
            block_size, C
        )
    elif method == "otsu":
        _, bin_img = cv2.threshold(
            img_blur, 0, 255,
            cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
        )
    elif method == "sauvola":
        # Sauvola via scikit-image if available
        from skimage.filters import threshold_sauvola
        thresh_s = threshold_sauvola(img_blur, window_size=block_size, k=0.4)
        bin_img = (img_blur < thresh_s).astype(np.uint8) * 255
    else:
        raise ValueError("method must be one of: adaptive, otsu, sauvola")

    # --- Step 3: Morphological cleanup ---
    kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_open, morph_open))
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (morph_close, morph_close))

    # Opening: remove small noise
    cleaned = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, kernel_open)

    # Closing: fill small holes in dots
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel_close)

    # --- Step 4: Optional top-hat (enhance embossing if faint) ---
    if apply_tophat:
        kernel_th = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        tophat = cv2.morphologyEx(img_gray, cv2.MORPH_TOPHAT, kernel_th)
        cleaned = cv2.bitwise_or(cleaned, (tophat > 0).astype(np.uint8) * 255)

    # --- Step 5: Optional small blob removal ---
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(cleaned, connectivity=8)
    mask = np.zeros_like(cleaned)
    for i in range(1, num_labels):  # skip background
        if stats[i, cv2.CC_STAT_AREA] >= min_blob_area:
            mask[labels == i] = 255

    return mask

def consolidate_braille_dots(binary_img):
    """
    Fix broken or irregular Braille dots before centroid detection.
    Returns cleaned binary image.
    """
    # Step 1. Close small holes inside each dot
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    closed = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel_close)

    # Step 2. Remove small specks between dots
    kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel_open)

    # Step 3. Optional: smooth edges slightly
    blurred = cv2.GaussianBlur(opened, (3, 3), 0)
    _, smooth_bin = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)

    return smooth_bin

def detect_braille_dots(
    binary_img,
    method="connected",      # "connected", "blob", or "log"
    min_area=5,              # adjust based on resolution (~π*r_min²)
    max_area=200,            # reject large merged blobs
    circularity_thresh=0.6,  # for blob detector
    inertia_thresh=0.5,      # for blob detector
    log_min_sigma=1,         # for LoG
    log_max_sigma=4,
    log_num_sigma=10,
    log_threshold=0.05
):
    """
    Detect Braille dot centroids in a preprocessed binary image.
    Returns list of (x, y) coordinates.
    """

    centroids = []

    # --- Option 1: Connected Components ---
    if method == "connected":
        num_labels, labels, stats, ctds = cv2.connectedComponentsWithStats(binary_img, connectivity=8)
        for i in range(1, num_labels):  # skip background
            area = stats[i, cv2.CC_STAT_AREA]
            if min_area <= area <= max_area:
                x, y = ctds[i]
                centroids.append((float(x), float(y)))

    # --- Option 2: OpenCV SimpleBlobDetector ---
    elif method == "blob":
        params = cv2.SimpleBlobDetector_Params()
        params.filterByArea = True
        params.minArea = min_area
        params.maxArea = max_area
        params.filterByCircularity = True
        params.minCircularity = circularity_thresh
        params.filterByInertia = True
        params.minInertiaRatio = inertia_thresh
        params.filterByConvexity = False

        detector = cv2.SimpleBlobDetector_create(params)
        keypoints = detector.detect(binary_img)
        centroids = [(kp.pt[0], kp.pt[1]) for kp in keypoints]

    # --- Option 3: Laplacian of Gaussian (LoG) ---
    elif method == "log":
        from skimage.feature import blob_log
        binary_float = binary_img.astype(np.float32) / 255.0
        blobs = blob_log(binary_float,
                         min_sigma=log_min_sigma,
                         max_sigma=log_max_sigma,
                         num_sigma=log_num_sigma,
                         threshold=log_threshold)
        # Each blob: (y, x, sigma)
        centroids = [(float(x), float(y)) for y, x, s in blobs]

    else:
        raise ValueError("method must be one of: 'connected', 'blob', or 'log'")

    return np.array(centroids, dtype=np.float32)

In [4]:
# Example usage:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
preprocessed = preprocess_braille_image(img_gray, method="sauvola")
# cv2.imshow('Before', img_gray)
# cv2.imshow('Preprocessed', binary)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
cv2.imwrite("braille_page_output_1.jpg",img)
binary = consolidate_braille_dots(preprocessed)
dots = detect_braille_dots(binary, method="connected", min_area=15, max_area=220)
print("Detected dots:", len(dots))
for (x, y) in dots:
    cv2.circle(img, (int(x), int(y)), 7, (0,0,255), -1)
# Store results
height, width, channels = img.shape
# Create a white image with the same size
output_image = np.ones((height, width, channels), dtype=np.uint8) * 255
for (x, y) in dots:
    cv2.circle(output_image, (int(x), int(y)), 4, (0,0,0), -1)
cv2.imwrite("braille_page_output_2.jpg",binary)
cv2.imwrite("braille_page_output_3.jpg",img)
cv2.imwrite("braille_page_output_4.jpg",output_image)

Detected dots: 281


True

In [5]:
def calculate_distances(dot_coordinates):
    """
    Calculate the average vertical (D_V), horizontal (D_H), and inter-cell horizontal (C_H) distances
    based on the given dot coordinates.
    
    Parameters:
    - dot_coordinates: List of (x, y) coordinates of the dots.
    
    Returns:
    - D_V: Average vertical distance.
    - D_H: Average horizontal distance.
    - C_H: Average inter-cell horizontal distance.
    """
    # Convert coordinates to numpy array for easier manipulation
    dots = np.array(dot_coordinates)
    
    # Calculate all pairwise vertical distances
    vertical_distances = []
    for i in range(len(dots)):
        for j in range(i + 1, len(dots)):
            if abs(dots[i][0] - dots[j][0]) < 10:  # Close enough in the horizontal direction
                vertical_distances.append(abs(dots[i][1] - dots[j][1]))
    
    D_V = np.mean(vertical_distances) if vertical_distances else 0
    
    # Calculate all pairwise horizontal distances
    horizontal_distances = []
    for i in range(len(dots)):
        for j in range(i + 1, len(dots)):
            if abs(dots[i][1] - dots[j][1]) < 10:  # Close enough in the vertical direction
                horizontal_distances.append(abs(dots[i][0] - dots[j][0]))
    
    D_H = np.mean(horizontal_distances) if horizontal_distances else 0
    
    # Sort the dots by their x-coordinate to estimate inter-cell distances
    dots_sorted_by_x = sorted(dots, key=lambda x: x[0])
    
    inter_cell_distances = []
    for i in range(1, len(dots_sorted_by_x)):
        # Horizontal distance between rightmost dot of one cell and leftmost dot of the next cell
        if abs(dots_sorted_by_x[i][1] - dots_sorted_by_x[i-1][1]) > D_V:  # Not part of the same cell vertically
            inter_cell_distances.append(abs(dots_sorted_by_x[i][0] - dots_sorted_by_x[i-1][0]))
    
    C_H = np.mean(inter_cell_distances) if inter_cell_distances else 0
    
    return D_V, D_H, C_H


def cluster_braille_dots(dot_coordinates, D_V, D_H, C_H):
    """
    Cluster the dots into braille cells based on proximity.
    
    Parameters:
    - dot_coordinates: List of (x, y) coordinates of dots.
    - D_V: Vertical distance between dots within a cell.
    - D_H: Horizontal distance between dots within a cell.
    - C_H: Horizontal distance between cells.
    
    Returns:
    - cells: List of clusters, each representing a braille cell.
    """
    
    # Use DBSCAN clustering to group the dots into cells
    X = np.array(dot_coordinates)
    
    # Calculate maximum distances between dots for clustering
    max_distance = max(D_H * 2, D_V * 3)  # Max allowed distance between dots in one cell
    
    # Apply DBSCAN clustering
    dbscan = DBSCAN(eps=max_distance, min_samples=1)
    labels = dbscan.fit_predict(X)
    
    # Group dots by cluster labels (cells)
    cells = {}
    for i, label in enumerate(labels):
        if label not in cells:
            cells[label] = []
        cells[label].append(X[i])
    
    return list(cells.values())

def normalize_and_extract_features(cells, D_V, D_H):
    """
    Normalize each cell to have consistent coordinates for the six dot positions and 
    return the coordinates of dots within each cell.
    
    Parameters:
    - cells: List of clusters, each representing a braille cell (set of coordinates).
    - D_V: Vertical distance between dots within a cell.
    - D_H: Horizontal distance between dots within a cell.
    
    Returns:
    - normalized_cells: List of normalized cells with coordinates for dots.
    """
    
    normalized_cells = []
    
    for cell in cells:
        # Initialize the six positions (dots 1-6) with coordinates (or None if not present)
        cell_coordinates = [None, None, None, None, None, None]  # [dot1, dot2, dot3, dot4, dot5, dot6]
        
        # Find the bounding box of the cell (min_x, min_y) to (max_x, max_y)
        min_x = min([dot[0] for dot in cell])
        min_y = min([dot[1] for dot in cell])
        max_x = max([dot[0] for dot in cell])
        max_y = max([dot[1] for dot in cell])
        
        # Normalize positions to fit in the 6-dot braille cell grid
        for dot in cell:
            x, y = dot
            if y - min_y < D_V:
                if x - min_x < D_H:
                    cell_coordinates[0] = dot  # Dot 1
                elif x - min_x < 2 * D_H:
                    cell_coordinates[1] = dot  # Dot 2
                else:
                    cell_coordinates[2] = dot  # Dot 3
            elif y - min_y < 2 * D_V:
                if x - min_x < D_H:
                    cell_coordinates[3] = dot  # Dot 4
                elif x - min_x < 2 * D_H:
                    cell_coordinates[4] = dot  # Dot 5
                else:
                    cell_coordinates[5] = dot  # Dot 6
        
        normalized_cells.append(cell_coordinates)
    
    return normalized_cells

def draw_bounding_boxes(image, cells, D_V, D_H):
    """
    Draw bounding boxes around each identified braille cell.
    
    Parameters:
    - image: The input image (OpenCV format).
    - cells: List of clusters, each representing a braille cell (set of coordinates).
    - D_V: Vertical distance between dots within a cell.
    - D_H: Horizontal distance between dots within a cell.
    
    Returns:
    - image_with_boxes: Image with bounding boxes drawn around each braille cell.
    """
    
    image_with_boxes = image.copy()
    
    for idx, cell in enumerate(cells):
        # Find the valid dots (coordinates) in the current braille cell
        valid_dots = [dot for dot in cell if dot is not None]
        
        if valid_dots:
            # Calculate the bounding box for the current braille cell
            min_x = min([dot[0] for dot in valid_dots])
            min_y = min([dot[1] for dot in valid_dots])
            max_x = max([dot[0] for dot in valid_dots])
            max_y = max([dot[1] for dot in valid_dots])

            # Debugging: print values of the bounding box coordinates
            print(f"Cell {idx}: min_x = {min_x}, min_y = {min_y}, max_x = {max_x}, max_y = {max_y}")

            # Check if the values are valid integers and inside image bounds
            if isinstance(min_x, int) and isinstance(min_y, int) and isinstance(max_x, int) and isinstance(max_y, int):
                # Draw a rectangle (bounding box) around the braille cell
                cv2.rectangle(image_with_boxes, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)
            else:
                print(f"Invalid bounding box coordinates: min_x={min_x}, min_y={min_y}, max_x={max_x}, max_y={max_y}")
    
    return image_with_boxes

In [6]:
# Usage
D_V, D_H, C_H = calculate_distances(dots)
cells = cluster_braille_dots(dots,D_V,D_H,C_H)
normalized_cells=normalize_and_extract_features(cells,D_V,D_H)
op= draw_bounding_boxes(output_image,normalized_cells, D_V, D_H)
cv2.imshow("Image with Boxes", op)
cv2.waitKey(0)
cv2.destroyAllWindows()



Cell 0: min_x = 119.72000122070312, min_y = 161.3265380859375, max_x = 1298.8775634765625, max_y = 294.2471923828125
Invalid bounding box coordinates: min_x=119.72000122070312, min_y=161.3265380859375, max_x=1298.8775634765625, max_y=294.2471923828125


In [7]:
# --- Step 1: Normalize Y positions into 3 row levels ---
def normalize_rows(centers, tolerance=3):
    """
    Normalize rows of Braille dots based on their y-coordinates. This function includes a tolerance 
    to treat closely spaced dots as part of the same row, even if their gap is smaller than the median gap.

    Args:
        centers (np.ndarray): 2D array of dot centroids with shape (N, 2), where N is the number of dots.
        tolerance (float): Maximum allowed y-gap between two dots to be considered part of the same row.
    
    Returns:
        row_levels (list): List of normalized row levels (y-values) that correspond to the rows of Braille dots.
    """
    ys = np.sort(centers[:, 1])  # Sort by y-coordinate (vertical positions of dots)
    y_gaps = np.diff(ys)  # Calculate the differences between consecutive y-values
    
    # Calculate the median y-gap as the threshold for row separation
    threshold = np.percentile(y_gaps, 50)
    
    row_levels = []
    temp = [ys[0]]
    
    for i, d in enumerate(y_gaps):
        if d > threshold and d > tolerance:  # Check for large gap or if gap exceeds tolerance
            row_levels.append(np.mean(temp))  # Add the mean of the previous row
            temp = [ys[i + 1]]  # Start a new row
        else:
            temp.append(ys[i + 1])  # Continue adding dots to the same row
    
    # Add the last row (mean of the last group of y-values)
    row_levels.append(np.mean(temp))
    return sorted(row_levels)

def plot_row_lines_on_image(image, centers):
    """
    Plots row lines on the input image at the y-positions determined by the normalize_rows function.
    The row lines are drawn in red.

    Args:
        image (np.ndarray): The image on which to draw the row lines.
        centers (np.ndarray): 2D array of dot centroids with shape (N, 2).
        tolerance (float): Maximum allowed y-gap between two dots to be considered part of the same row.
    
    Returns:
        np.ndarray: Image with the row lines plotted in red.
    """
    # Step 1: Normalize the y-positions of the centers to determine row levels
    row_levels = normalize_rows(centers)
    
    # Step 2: Draw horizontal red lines on the image at the row levels
    for row in row_levels:
        # Draw a red horizontal line at y = row
        cv2.line(image, (0, int(row)), (image.shape[1], int(row)), (0, 0, 255), 2)  # Red color in BGR
    
    return image

# output_image = plot_row_lines_on_image(img, dots)

# Show the resulting image
# cv2.imshow('Image with Row Lines', output_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
# Calculate pairwise Euclidean distances
pairwise_distances = cdist(dots, dots, metric='euclidean')
# Calculate all pairwise x and y differences
x_diffs = np.abs(dots[:, 0][:, None] - dots[:, 0])
y_diffs = np.abs(dots[:, 1][:, None] - dots[:, 1])

# Exclude zero distances (self-comparisons)
nonzero_x = x_diffs[x_diffs > 0]
nonzero_y = y_diffs[y_diffs > 0]

min_x, max_x = nonzero_x.min(), nonzero_x.max()
min_y, max_y = nonzero_y.min(), nonzero_y.max()

print(f"Minimum horizontal (x) distance: {min_x:.3f}")
print(f"Maximum horizontal (x) distance: {max_x:.3f}")
print(f"Minimum vertical (y) distance: {min_y:.3f}")
print(f"Maximum vertical (y) distance: {max_y:.3f}")


def kmeans_clusters(dots, n_clusters=6, image=None, show_centers=False):
    """
    Apply K-means clustering to Braille dot centroids and visualize results using OpenCV.
    
    Args:
        dots: np.ndarray of shape (N, 2)
        n_clusters: int – number of K-means clusters
        image: optional grayscale or BGR background image
        show_centers: bool – whether to draw yellow cluster centers
    """
    if len(dots) == 0:
        print("No dots provided.")
        return
    
    # --- 1️⃣ Run K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(dots)

    # --- 2️⃣ Prepare visualization canvas
    if image is not None:
        vis = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR) if len(image.shape) == 2 else image.copy()
    else:
        w = int(np.max(dots[:,0]) + 20)
        h = int(np.max(dots[:,1]) + 20)
        vis = np.ones((h, w, 3), dtype=np.uint8) * 30  # dark background

    # --- 3️⃣ Assign distinct colors per cluster
    rng = np.random.default_rng(42)
    palette = (rng.integers(40, 255, size=(n_clusters, 3))).astype(int)

    # --- 4️⃣ Draw each dot by cluster color
    for (x, y), label in zip(dots, labels):
        color = tuple(int(c) for c in palette[label])
        cv2.circle(vis, (int(x), int(y)), 8, color, -1)
    
    # --- 4.1️⃣ Draw bounding boxes around each cluster
    for label in range(n_clusters):
        cluster_points = dots[labels == label]
        if len(cluster_points) == 0:
            continue
        x1, y1 = np.min(cluster_points, axis=0).astype(int)
        x2, y2 = np.max(cluster_points, axis=0).astype(int)
        # cv2.rectangle(vis, (x1-15, y1-15), (x2+15, y2+15), (0, 255, 0), 2)
        # print(123)
        # cropped_image = vis[y1-15:y2+15, x1-15:x2+15]
        # img_gray_loop = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        # preprocessed_loop = preprocess_braille_image(img_gray_loop, method="sauvola")
        # binary_loop = consolidate_braille_dots(preprocessed_loop)
        # dots_loop = detect_braille_dots(binary_loop, method="connected", min_area=25, max_area=220)
        # bit_vectors, row_levels, col_gap, cells = process_braille(cluster_points)
        # visualize_braille_segmentation(image, cluster_points, row_levels, cells, col_gap)        

    # --- 5️⃣ Optionally draw cluster centers
    if show_centers:
        for (cx, cy) in kmeans.cluster_centers_:
            cv2.drawMarker(vis, (int(cx), int(cy)), (0, 255, 255),
                           markerType=cv2.MARKER_CROSS, markerSize=14, thickness=2)

    # --- 6️⃣ Show image
    # cv2.imshow(f"K-means ({n_clusters} clusters)", vis)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    cv2.imwrite("braille_page_output_6.jpg",vis)

    return labels


labels = kmeans_clusters(dots, n_clusters=6, image=img)
for i, lbl in enumerate(labels):
    print(f"Dot {i+1}: Cluster {lbl}")

Minimum horizontal (x) distance: 0.016
Maximum horizontal (x) distance: 1330.755
Minimum vertical (y) distance: 0.000
Maximum vertical (y) distance: 359.539
Dot 1: Cluster 2
Dot 2: Cluster 2
Dot 3: Cluster 5
Dot 4: Cluster 1
Dot 5: Cluster 1
Dot 6: Cluster 1
Dot 7: Cluster 1
Dot 8: Cluster 2
Dot 9: Cluster 4
Dot 10: Cluster 4
Dot 11: Cluster 4
Dot 12: Cluster 4
Dot 13: Cluster 4
Dot 14: Cluster 0
Dot 15: Cluster 0
Dot 16: Cluster 0
Dot 17: Cluster 0
Dot 18: Cluster 5
Dot 19: Cluster 5
Dot 20: Cluster 5
Dot 21: Cluster 1
Dot 22: Cluster 1
Dot 23: Cluster 2
Dot 24: Cluster 0
Dot 25: Cluster 0
Dot 26: Cluster 0
Dot 27: Cluster 5
Dot 28: Cluster 5
Dot 29: Cluster 4
Dot 30: Cluster 4
Dot 31: Cluster 2
Dot 32: Cluster 2
Dot 33: Cluster 4
Dot 34: Cluster 4
Dot 35: Cluster 0
Dot 36: Cluster 0
Dot 37: Cluster 0
Dot 38: Cluster 5
Dot 39: Cluster 5
Dot 40: Cluster 1
Dot 41: Cluster 1
Dot 42: Cluster 2
Dot 43: Cluster 2
Dot 44: Cluster 2
Dot 45: Cluster 2
Dot 46: Cluster 2
Dot 47: Cluster 5
Dot 48

In [9]:
# Will use for next steps


# import numpy as np
# import cv2
# from sklearn.cluster import KMeans, DBSCAN

# def localized_kmeans_clustering(dots, image=None, show=True):
#     """
#     Cluster Braille dots into local groups (cells) using DBSCAN + localized KMeans.
#     Ensures KMeans only runs on close dots (prevents cross-letter mixing).
#     """
#     if len(dots) == 0:
#         print("No dots provided.")
#         return []

#     dots = np.array(dots, dtype=np.float32)

#     # --- Step 1️⃣: Estimate typical spacing
#     xs, ys = dots[:, 0], dots[:, 1]
#     dx = np.median(np.diff(np.sort(xs))) if len(xs) > 1 else 10
#     dy = np.median(np.diff(np.sort(ys))) if len(ys) > 1 else 10
#     eps = np.mean([dx, dy]) * 40  # local region size

#     # --- Step 2️⃣: Use DBSCAN to find local neighborhoods
#     y_scale=2
#     dots_scaled = dots.copy()
#     dots_scaled[:, 1] *= y_scale
#     db = DBSCAN(eps=eps, min_samples=1).fit(dots_scaled)
#     labels = db.labels_

#     cell_clusters = []
#     for lbl in sorted(set(labels)):
#         cluster_pts = dots[labels == lbl]
#         if len(cluster_pts) == 0:
#             continue

#         # --- Step 3️⃣: Run KMeans *locally* only if >6 dots
#         if len(cluster_pts) > 6:
#             n_sub = int(np.ceil(len(cluster_pts) / 6))
#             kmeans = KMeans(n_clusters=n_sub, random_state=42, n_init=10)
#             sub_labels = kmeans.fit_predict(cluster_pts)
#             for sub_lbl in np.unique(sub_labels):
#                 pts = cluster_pts[sub_labels == sub_lbl]
#                 cell_clusters.append(pts)
#         else:
#             cell_clusters.append(cluster_pts)

#     # --- Step 4️⃣: Visualization (optional)
#     if show:
#         if image is not None:
#             vis = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR) if len(image.shape)==2 else image.copy()
#         else:
#             w, h = int(np.max(xs) + 20), int(np.max(ys) + 20)
#             vis = np.ones((h, w, 3), np.uint8) * 30

#         rng = np.random.default_rng(42)
#         colors = rng.integers(60, 255, (len(cell_clusters), 3))

#         for i, cluster in enumerate(cell_clusters):
#             color = tuple(int(c) for c in colors[i])
#             for (x, y) in cluster:
#                 cv2.circle(vis, (int(x), int(y)), 7, color, -1)

#             x1, y1 = np.min(cluster, axis=0).astype(int)
#             x2, y2 = np.max(cluster, axis=0).astype(int)
#             cv2.rectangle(vis, (x1-4, y1-4), (x2+4, y2+4), color, 2)
        
#         # Draw bounding boxes for each cluster
#         for cluster in cell_clusters:
#             x1, y1 = np.min(cluster, axis=0).astype(int)
#             x2, y2 = np.max(cluster, axis=0).astype(int)
#             cv2.rectangle(vis, (x1-3, y1-3), (x2+3, y2+3), (0, 255, 0), 2)

#         cv2.imshow("Localized KMeans Clustering", vis)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

#     return cell_clusters

# cells = localized_kmeans_clustering(dots, image=output_image)
# print(f"Detected {len(cells)} Braille cells.")


In [10]:
# Will use for next steps

# import numpy as np
# from scipy.spatial.distance import pdist
# from sklearn.neighbors import KNeighborsClassifier
# import cv2

# def reconstruct_braille_cell(centroids, visualize=True, image=None):
#     """
#     Reconstruct a Braille cell from dot centroids using geometric normalization + ML (KNN).
    
#     Args:
#         centroids : np.ndarray of shape (N, 2) – detected dot centers
#         visualize : bool – whether to visualize on OpenCV image
#         image     : optional image (grayscale or BGR) for overlay
    
#     Returns:
#         mat        : 3×2 np.uint8 matrix (1 = dot present, 0 = absent)
#         positions  : list of predicted Braille dot indices (1–6)
#     """
#     if centroids is None or len(centroids) == 0:
#         print("No centroids provided.")
#         return np.zeros((3,2), dtype=np.uint8), []

#     centroids = np.array(centroids, dtype=np.float32)

#     # --- Step 1️⃣: Estimate scale (d_unit)
#     if len(centroids) > 1:
#         D = pdist(centroids)
#         d_unit = np.median(D)
#     else:
#         d_unit = 1.0  # fallback for single dot

#     # --- Step 2️⃣: Normalize coordinate space
#     centroids_norm = centroids - centroids.min(axis=0)
#     centroids_norm /= (d_unit + 1e-6)

#     # --- Step 3️⃣: KNN classifier for canonical Braille layout
#     X_train = np.array([
#         [0.0, 0.0], [0.0, 1.0], [0.0, 2.0],      # left column dots 1–3
#         [1.6, 0.0], [1.6, 1.0], [1.6, 2.0]       # right column dots 4–6
#     ])
#     y_train = np.array([1, 2, 3, 4, 5, 6])

#     knn = KNeighborsClassifier(n_neighbors=1)
#     knn.fit(X_train, y_train)

#     positions = knn.predict(centroids_norm)

#     # --- Step 4️⃣: Form 2×3 Braille matrix
#     mat = np.zeros((3, 2), dtype=np.uint8)
#     mapping = {1:(0,0), 2:(1,0), 3:(2,0), 4:(0,1), 5:(1,1), 6:(2,1)}
#     for p in positions:
#         r, c = mapping[p]
#         mat[r, c] = 1

#     # --- Optional visualization
#     if visualize:
#         if image is not None:
#             vis = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR) if len(image.shape)==2 else image.copy()
#         else:
#             w = int(np.max(centroids[:,0]) + 40)
#             h = int(np.max(centroids[:,1]) + 40)
#             vis = np.ones((h, w, 3), np.uint8) * 40

#         # Draw dots
#         for (x, y), p in zip(centroids, positions):
#             cv2.circle(vis, (int(x), int(y)), 8, (0, 255, 255), -1)
#             cv2.putText(vis, str(p), (int(x)+10, int(y)-5), cv2.FONT_HERSHEY_SIMPLEX, 
#                         0.6, (0, 255, 0), 2, cv2.LINE_AA)

#         # Bounding box
#         x1, y1 = np.min(centroids, axis=0).astype(int)
#         x2, y2 = np.max(centroids, axis=0).astype(int)
#         cv2.rectangle(vis, (x1-5, y1-5), (x2+5, y2+5), (0, 200, 0), 2)

#         cv2.imshow("Braille Cell Reconstruction", vis)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

#     return mat, positions


# # --- 🧩 Example usage ---
# # Simulate a Braille cell (slightly skewed, missing dots possible)

# mat, pos = reconstruct_braille_cell(dots, visualize=True, image=img)

# print("\nPredicted dot positions (1–6):", pos)
# print("\nBraille Matrix (3x2):\n", mat)


In [11]:
def cluster_and_visualize_braille(centroids):
    # Assume centroids is a list of [x, y] lists or array
    centroids = np.array(centroids)
    
    # Sort by Y (top to bottom)
    centroids = centroids[np.argsort(centroids[:, 1])]
    
    # Compute median vertical and horizontal spacings
    y_diffs = np.abs(np.diff(np.sort(centroids[:, 1])))
    dy = np.median(y_diffs[y_diffs > 0]) if np.any(y_diffs > 0) else 1.0  # Fallback to avoid zero
    
    x_diffs = np.abs(np.diff(np.sort(centroids[:, 0])))
    dx = np.median(x_diffs[x_diffs > 0]) if np.any(x_diffs > 0) else 1.0  # Fallback to avoid zero
    
    # Tolerance for slight tilt (3%)
    tolerance = 1.2
    
    # Thresholds with tolerance
    vertical_thresh = dy * 1.1 * tolerance  # Adjusted for same column / vertical closeness
    horizontal_thresh = dy * 3.5 * tolerance  # Adjusted for horizontal gap between cells
    
    # Row grouping: group dots into text lines using vertical threshold * 3 with tolerance
    line_thresh = vertical_thresh * 5 * tolerance  # Increased for potential tilt variation
    rows = []
    for dot in centroids:
        added = False
        for row in rows:
            if abs(row['mean_y'] - dot[1]) < line_thresh:
                row['dots'].append(dot)
                row['mean_y'] = np.mean([d[1] for d in row['dots']])
                added = True
                break
        if not added:
            rows.append({'dots': [dot], 'mean_y': dot[1]})
    
    # For each row, sort dots by x and group into cells
    all_cells = []
    for row in rows:
        # Sort dots in row by x (left to right)
        row['dots'] = sorted(row['dots'], key=lambda d: d[0])
        
        # Group into cells based on horizontal gaps
        cells = []
        if len(row['dots']) > 0:
            current = [row['dots'][0]]
            for i in range(1, len(row['dots'])):
                if abs(row['dots'][i][0] - row['dots'][i-1][0]) < horizontal_thresh:
                    current.append(row['dots'][i])
                else:
                    if len(current) <= 6:  # Braille cell <=6 dots
                        cells.append(current)
                    current = [row['dots'][i]]
            if len(current) <= 6:
                cells.append(current)
        row['cells'] = cells
        all_cells.extend(cells)
    
    # Visualization with OpenCV
    if len(centroids) == 0:
        print("No centroids provided.")
        return
    
    min_x = np.min(centroids[:, 0])
    max_x = np.max(centroids[:, 0])
    min_y = np.min(centroids[:, 1])
    max_y = np.max(centroids[:, 1])
    
    # Create blank image with padding
    padding = 50
    width = int(max_x - min_x + 2 * padding)
    height = int(max_y - min_y + 2 * padding)
    img = np.ones((height, width, 3), np.uint8) * 255  # White background
    
    # Draw dots (shifted)
    dot_radius = 5
    for dot in centroids:
        x_sh = int(dot[0] - min_x + padding)
        y_sh = int(dot[1] - min_y + padding)
        cv2.circle(img, (x_sh, y_sh), dot_radius, (0, 0, 255), -1)  # Red dots
    
    # Draw bounding boxes around cells
    box_padding = 10
    for cell in all_cells:
        if len(cell) > 0:
            cell_arr = np.array(cell)
            min_cx = np.min(cell_arr[:, 0])
            max_cx = np.max(cell_arr[:, 0])
            min_cy = np.min(cell_arr[:, 1])
            max_cy = np.max(cell_arr[:, 1])
            
            x1 = int(min_cx - min_x + padding - box_padding)
            y1 = int(min_cy - min_y + padding - box_padding)
            x2 = int(max_cx - min_x + padding + box_padding)
            y2 = int(max_cy - min_y + padding + box_padding)
            
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green boxes
    
    # Save the image
    cv2.imshow('braille_cells.png', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # print("Visualization saved as 'braille_cells.png'")
    print(f"Found {len(all_cells)} Braille cells.")

# Example usage (replace with your centroids)
# sample_centroids = [
#     [10, 10], [30, 10], [10, 30], [30, 30],  # Cell 1
#     [70, 10], [90, 10], [70, 30]  # Cell 2, incomplete
# ]
cluster_and_visualize_braille(dots)

Found 281 Braille cells.


In [12]:
"""
braille_segment.py

Usage:
    - Provide a list of centroids: list of (x, y) tuples (float or int).
    - Optionally pass an image path to draw boxes on the source image.
    - This script uses OpenCV (cv2) and numpy. sklearn is optional (for DBSCAN fallback).

Outputs:
    - Returns rows/cells structure and saves a visualization image (if visualize=True).
"""

import math
import numpy as np
import cv2

# Optional imports
try:
    from sklearn.cluster import DBSCAN
    SKLEARN_AVAILABLE = True
except Exception:
    SKLEARN_AVAILABLE = False

def deskew_by_pca(pts):
    """
    Compute rotation angle from PCA and rotate points to make text horizontal.
    Returns rotated_points (Nx2), angle_radians, rotation_matrix, centroid
    """
    pts = np.asarray(pts, dtype=float)
    centroid = pts.mean(axis=0)
    centered = pts - centroid
    # SVD (PCA)
    U, S, Vt = np.linalg.svd(centered, full_matrices=False)
    # first principal component direction:
    pc1 = Vt[0]
    angle = math.atan2(pc1[1], pc1[0])  # angle of major axis wrt x-axis
    # We want rows horizontal -> rotate by -angle if |angle| > small threshold
    c = math.cos(-angle)
    s = math.sin(-angle)
    R = np.array([[c, -s],[s, c]])
    rotated = (centered @ R.T) + centroid
    return rotated, angle, R, centroid

def estimate_spacings(centroids, tolerance_multiplier=1.03):
    """
    Robust estimation of vertical (dy) and horizontal (dx) spacing.
    Returns dy, dx (both floats).
    """
    pts = np.asarray(centroids, dtype=float)
    ys = np.sort(pts[:,1])
    xs = np.sort(pts[:,0])

    if len(ys) < 2:
        dy = 1.0
    else:
        dy_diffs = np.abs(np.diff(ys))
        # filter out zeros
        dy_diffs = dy_diffs[dy_diffs > 0]
        dy = float(np.median(dy_diffs)) if dy_diffs.size > 0 else max(1.0, np.ptp(ys)/max(1, len(ys)))
    if len(xs) < 2:
        dx = 1.0
    else:
        dx_diffs = np.abs(np.diff(xs))
        dx_diffs = dx_diffs[dx_diffs > 0]
        dx = float(np.median(dx_diffs)) if dx_diffs.size > 0 else max(1.0, np.ptp(xs)/max(1, len(xs)))

    # apply tolerance multiplier (3% as requested)
    dy *= tolerance_multiplier
    dx *= tolerance_multiplier

    return dy, dx

def group_rows(pts, vertical_thresh):
    """
    Groups points into rows based on Y proximity.
    Input pts: Nx2 array (x,y)
    Returns list of rows: each row is dict with keys 'dots' (list of (x,y)), 'mean_y'
    """
    rows = []
    for dot in pts:
        x,y = float(dot[0]), float(dot[1])
        added = False
        for row in rows:
            if abs(row['mean_y'] - y) < vertical_thresh * 3:   # using the *3 as suggested
                row['dots'].append((x,y))
                row['mean_y'] = float(np.mean([d[1] for d in row['dots']]))
                added = True
                break
        if not added:
            rows.append({'dots': [(x,y)], 'mean_y': y})
    # sort rows by mean_y (top to bottom)
    rows = sorted(rows, key=lambda r: r['mean_y'])
    return rows

def group_cells_in_row(row, horizontal_thresh):
    """
    Given a row dict with 'dots' list (x,y), group into cells left-to-right using horizontal gaps.
    Returns list of cells (each cell is a list of (x,y))
    """
    dots = sorted(row['dots'], key=lambda d: d[0])
    if not dots:
        return []
    cells = []
    current = [dots[0]]
    for i in range(1, len(dots)):
        if abs(dots[i][0] - dots[i-1][0]) < horizontal_thresh:
            current.append(dots[i])
        else:
            cells.append(current)
            current = [dots[i]]
    cells.append(current)
    return cells

def fallback_dbscan(pts, eps):
    """
    Use DBSCAN on the raw points when row/col heuristics fail.
    Returns list of clusters: each cluster is a list of (x,y)
    """
    if not SKLEARN_AVAILABLE:
        # If sklearn not available, return single cluster
        return [list(map(tuple, pts))]
    db = DBSCAN(eps=eps, min_samples=1).fit(pts)
    labels = db.labels_
    clusters = []
    for lbl in sorted(set(labels)):
        cluster_pts = pts[labels == lbl]
        clusters.append([tuple(p) for p in cluster_pts])
    # sort clusters left-to-right by mean x
    clusters = sorted(clusters, key=lambda c: np.mean([p[0] for p in c]))
    return clusters

def make_bounding_box_for_cell(cell, padding):
    """
    cell: list of (x,y) points.
    padding: scalar to add to each side.
    Returns (x1,y1,x2,y2) as ints.
    """
    arr = np.array(cell)
    x1 = int(np.floor(arr[:,0].min() - padding))
    x2 = int(np.ceil(arr[:,0].max() + padding))
    y1 = int(np.floor(arr[:,1].min() - padding))
    y2 = int(np.ceil(arr[:,1].max() + padding))
    return (x1,y1,x2,y2)

def segment_braille_cells(centroids,
                          image_path=None,
                          visualize=True,
                          deskew=True,
                          tolerance_pct=0.03,
                          dbscan_fallback=True,
                          out_image_path="braille_cells_vis.png"):
    """
    Main function.
    centroids: list of (x,y)
    image_path: optional path to original image. If None, a blank canvas is used.
    visualize: whether to write an output image (OpenCV).
    deskew: whether to apply PCA deskew before grouping.
    tolerance_pct: decimal (0.03 -> 3%) applied to thresholds.
    dbscan_fallback: if True and a row grouping looks bad, run DBSCAN.
    Returns: dict with keys:
        - 'rows' : list of rows, each with 'mean_y', 'cells' (list of cell lists)
        - 'cells_bboxes' : list of (x1,y1,x2,y2) in **original coordinate space**
    """
    if len(centroids) == 0:
        return {'rows': [], 'cells_bboxes': []}

    # Convert to ndarray
    pts = np.asarray(centroids, dtype=float)

    # Deskew if requested
    rotated_pts = pts.copy()
    angle = 0.0
    R = np.eye(2)
    centroid_for_rot = np.array([0.,0.])
    if deskew:
        rotated_pts, angle, R, centroid_for_rot = deskew_by_pca(pts)
        # if angle is tiny, we still keep it — thresholds are tolerant
        # rotated_pts is in same coordinate space but rotated so rows are horizontal

    # Estimate spacings using rotated coords
    tolerance_multiplier = 1.0 + float(tolerance_pct)
    dy, dx = estimate_spacings(rotated_pts, tolerance_multiplier=tolerance_multiplier)
    # According to your heuristics:
    vertical_thresh = dy * 18  # dots within same column
    horizontal_thresh = dy * 8  # gap to next cell

    # Row grouping on rotated coords (so Y alignment is near horizontal)
    rows = group_rows(rotated_pts, vertical_thresh)

    # If rows look suspicious (too few rows or large variance), consider DBSCAN fallback
    # We'll mark suspicious if there is a row where the y-range of its dots > vertical_thresh*4
    suspicious = False
    for row in rows:
        if len(row['dots']) > 1:
            ys = [d[1] for d in row['dots']]
            if (max(ys) - min(ys)) > (vertical_thresh * 4):
                suspicious = True
                break

    cells_bboxes = []
    structured_rows = []
    if suspicious and dbscan_fallback:
        # run DBSCAN on rotated points
        eps = dy * 2.2 * tolerance_multiplier
        clusters = fallback_dbscan(rotated_pts, eps=eps)
        # treat each cluster as a cell
        for i, cluster in enumerate(clusters):
            # compute bbox in rotated space, then map back to original space
            padding = dy * 0.6
            bb = make_bounding_box_for_cell(cluster, padding)
            # convert bbox corners back to original using inverse rotation
            x1,y1,x2,y2 = bb
            corners = np.array([[x1,y1],[x2,y2]])
            # undo rotation: rotated_pts = (original - centroid)*R^T + centroid
            # so original = ((rotated - centroid) @ (R.T)^{-1}) + centroid
            # Since R is orthonormal, inverse is R.T
            R_inv = R.T
            orig_corners = ((corners - centroid_for_rot) @ R_inv.T) + centroid_for_rot
            x1o, y1o = orig_corners[0].astype(int)
            x2o, y2o = orig_corners[1].astype(int)
            # normalize
            x1o, x2o = min(x1o,x2o), max(x1o,x2o)
            y1o, y2o = min(y1o,y2o), max(y1o,y2o)
            cells_bboxes.append((x1o, y1o, x2o, y2o))
        # pack into structured_rows as single-row clusters
        structured_rows = []
        for c in clusters:
            structured_rows.append({
                'mean_y': float(np.mean([p[1] for p in c])),
                'cells': [c]
            })
    else:
        # Normal geometric grouping
        for row_idx, row in enumerate(rows):
            cells = group_cells_in_row(row, horizontal_thresh)
            structured_rows.append({'mean_y': row['mean_y'], 'cells': cells})
            # compute bboxes (map them back to original coordinate space if deskew used)
            for cell in cells:
                padding = dy * 0.6
                bb = make_bounding_box_for_cell(cell, padding)  # in rotated space
                x1,y1,x2,y2 = bb
                corners = np.array([[x1,y1],[x2,y2]])
                # undo rotation if deskew applied
                if deskew:
                    R_inv = R.T
                    orig_corners = ((corners - centroid_for_rot) @ R_inv.T) + centroid_for_rot
                    x1o, y1o = orig_corners[0].astype(int)
                    x2o, y2o = orig_corners[1].astype(int)
                else:
                    x1o, y1o, x2o, y2o = int(x1), int(y1), int(x2), int(y2)
                x1o, x2o = min(x1o,x2o), max(x1o,x2o)
                y1o, y2o = min(y1o,y2o), max(y1o,y2o)
                cells_bboxes.append((x1o, y1o, x2o, y2o))

    # Visualization
    if visualize:
        # If an image path is provided, try to load it. Otherwise make blank canvas scaled to points.
        # if image_path:
            # img = cv2.imread(image_path)
        img = image_path
        #     if img is None:
        #         print("Warning: failed to load image_path. Creating blank canvas instead.")
        #         image_path = None
        # if image_path is None:
        #     # create blank white canvas sized to points extents
        #     all_x = pts[:,0]
        #     all_y = pts[:,1]
        #     w = int(max(300, np.ceil(all_x.max() - all_x.min() + 200)))
        #     h = int(max(300, np.ceil(all_y.max() - all_y.min() + 200)))
        #     img = 255 * np.ones((h, w, 3), dtype=np.uint8)
        #     # shift points so they're visible (add padding)
        #     pad_x = int(100 - all_x.min())
        #     pad_y = int(100 - all_y.min())
        #     # also apply to bbox coordinates after creation below by shifting them
        #     shift = (pad_x, pad_y)
        #     shift_needed = True
        # else:
        shift = (0,0)
        shift_needed = False

        # draw original dots for reference (in red)
        for (x,y) in pts:
            cx = int(round(x + shift[0]))
            cy = int(round(y + shift[1]))
            cv2.circle(img, (cx, cy), 3, (0,0,255), -1)  # red dots

        # draw bounding boxes (blue) and label them
        for idx, bbox in enumerate(cells_bboxes):
            x1,y1,x2,y2 = bbox
            x1 += shift[0]; x2 += shift[0]; y1 += shift[1]; y2 += shift[1]
            # clamp
            x1 = max(0, x1); y1 = max(0, y1)
            x2 = min(img.shape[1]-1, x2); y2 = min(img.shape[0]-1, y2)
            cv2.rectangle(img, (x1,y1), (x2,y2), (255,0,0), 2)  # blue box
            cv2.putText(img, f"{idx+1}", (x1+3, y1+14), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)

        cv2.imshow(out_image_path, img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        print(f"Visualization written to: {out_image_path}")

    result = {
        'rows': structured_rows,
        'cells_bboxes': cells_bboxes,
        'dy': dy,
        'dx': dx,
        'angle_radians': angle
    }
    return result


if __name__ == "__main__":
    # ---------- Example usage ----------
    # Replace 'centroids' with your centroid list and optionally set image_path
    # example_centroids = [
    #     # a small mock example: two rows, three cells per row (various dots)
    #     (50, 50), (50, 70), (50, 90),       # first column, three vertical dots
    #     (110, 50), (110, 70),               # second column, two dots
    #     (170, 50), (170, 70), (170, 90),    # third column
    #     (50, 150), (50,170),                # second row first cell
    #     (110,150), (110,170), (110,190),
    #     (170,150),                          # ...
    # ]
    # # If you want to overlay on an actual high-res image of your scanned Braille page, set image_path
    # image_path = None  # e.g., "scan.jpg"

    res = segment_braille_cells(dots, image_path=img,
                                visualize=True, deskew=True,
                                tolerance_pct=0.03,
                                dbscan_fallback=True,
                                out_image_path="braille_cells_visualization.png")

    print("Detected cells (bboxes):")
    for i, bbox in enumerate(res['cells_bboxes'], 1):
        print(f"  Cell {i}: bbox={bbox}")
    print("dy (estimated vertical spacing):", res['dy'])
    print("angle (radians) used to deskew:", res['angle_radians'])


Visualization written to: braille_cells_visualization.png
Detected cells (bboxes):
  Cell 1: bbox=(np.int64(23), np.int64(57), np.int64(25), np.int64(59))
  Cell 2: bbox=(np.int64(79), np.int64(406), np.int64(81), np.int64(408))
  Cell 3: bbox=(np.int64(76), np.int64(368), np.int64(78), np.int64(370))
  Cell 4: bbox=(np.int64(73), np.int64(286), np.int64(75), np.int64(288))
  Cell 5: bbox=(np.int64(73), np.int64(246), np.int64(75), np.int64(248))
  Cell 6: bbox=(np.int64(72), np.int64(227), np.int64(74), np.int64(229))
  Cell 7: bbox=(np.int64(72), np.int64(209), np.int64(74), np.int64(211))
  Cell 8: bbox=(np.int64(71), np.int64(165), np.int64(75), np.int64(166))
  Cell 9: bbox=(np.int64(72), np.int64(130), np.int64(74), np.int64(131))
  Cell 10: bbox=(np.int64(95), np.int64(366), np.int64(97), np.int64(369))
  Cell 11: bbox=(np.int64(90), np.int64(305), np.int64(93), np.int64(306))
  Cell 12: bbox=(np.int64(91), np.int64(287), np.int64(93), np.int64(289))
  Cell 13: bbox=(np.int64(90